In [1]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
from bs4 import BeautifulSoup
from requests.exceptions import ConnectTimeout, ReadTimeout, RequestException
import itertools



In [2]:
# this section of code make a request and permits up to max_retries 
def get_page(url, max_retries=3):
    retries = 0
    while retries < max_retries:
        try:
            # Attempt to make the HTTP GET request with a specified timeout
            response = requests.get(url, timeout=2)  # adding a sensible timeout
            # If the request is successful, break out of the loop
            if response.status_code == 200:
                return response.text
            if response.status_code == 404:
                return None , None
        except (ConnectTimeout, ReadTimeout) as e:
            # Print an error message showing the retry attempt
            print(f"Attempt {retries + 1} of {max_retries} failed with timeout. Retrying...")
            retries += 1
            if retries == max_retries:
                print("Max retries exceeded. Failing...")
                raise  # Re-raise the last exception after final attempt
        except RequestException as e:
            print(f"Attempt {retries + 1} failed with a request exception: {e}. Retrying...")
            retries += 1
            if retries == max_retries:
                print("Max retries exceeded with request exceptions. Failing...")
                raise
        except: 
            return None, None

    # If all retries are exhausted without a return, it indicates a failure
    print("Request failed after maximum retries.")
    return None, None


In [3]:
def get_county_abbr():
    # Example URL
    url = "https://fred.stlouisfed.org/categories/29898"

    # Fetch the webpage
    response = get_page(url)


    # Parse the HTML
    soup = BeautifulSoup(response, 'html.parser')

    # Assuming county_tags are correctly selected from the parsed HTML
    county_tags = soup.find_all('a', href=lambda href: href and "categories" in href)

    # Process and collect county names
    counties_nospace = [tag.get_text().strip().replace(" ", "")[:4].upper() for tag in county_tags if 'County' in tag.get_text()]

    # Print results
    print(counties_nospace)
    return counties_nospace


In [4]:
# this section of code make a request and permits up to max_retries 
def make_request_with_retries(url, max_retries=3):
    retries = 0
    while retries < max_retries:
        try:
            # Attempt to make the HTTP GET request with a specified timeout
            response = requests.get(url, timeout=2)  # adding a sensible timeout
            # If the request is successful, break out of the loop
            if response.status_code == 200:
                if response:
                    print(f"Request succeeded for {url}")
                    # Parse the HTML content
                    soup = BeautifulSoup(response.text, 'html.parser')
                    # Find the <title> tag
                    title_tag = soup.find('title')
                    # Extract the text from the <title> tag
                    title_text = title_tag.text if title_tag else None
                    # Add the title text to the list if it exists
                    if title_text:
                        # Split the string to isolate the part before the first '|'
                        main_part = title_text.split('|')[0].strip().replace("Resident Population in ", "")
                        # Extract the county name; it's always followed by ", TX"
                        county = main_part.split(", TX")[0].split('for ')[-1].strip()
                        # Extract the series ID; it's always within parentheses
                        series_id = main_part.split('(')[-1].split(')')[0].strip()
                        display (f' county  {county} series id {series_id}')
                return county,series_id
            if response.status_code == 404:
                return None , None
        except (ConnectTimeout, ReadTimeout) as e:
            # Print an error message showing the retry attempt
            print(f"Attempt {retries + 1} of {max_retries} failed with timeout. Retrying...")
            retries += 1
            if retries == max_retries:
                print("Max retries exceeded. Failing...")
                raise  # Re-raise the last exception after final attempt
        except RequestException as e:
            print(f"Attempt {retries + 1} failed with a request exception: {e}. Retrying...")
            retries += 1
            if retries == max_retries:
                print("Max retries exceeded with request exceptions. Failing...")
                raise
        except: 
            return None, None

    # If all retries are exhausted without a return, it indicates a failure
    print("Request failed after maximum retries.")
    return None, None


In [5]:
def get_pop_series_id (county_abbr):
   
    for idx in range (1,10,2):
         series_id = f'TX{county_abbr}{idx}POP'
         # display (series_id)

    return series_id, 'no'

In [6]:
county_abbrs = get_county_abbr()
counties = []
pop_ids = []
for county_abbr in county_abbrs:
    for idx in itertools.chain(range(1, 10, 2), range(0, 10, 2)):
        series_id = f'TX{county_abbr}{idx}POP'
        if pop_ids and pop_ids[-1] == series_id: 
            series_id = f'TX{county_abbr}{idx+2}POP'
            display (f'this is the dup abbr code {pop_ids[-1]}')
            # break
        # display (series_id)
        # for index in range(1, 520):  # Adjusted range to Python syntax
        try:

            url = f'https://fred.stlouisfed.org/series/{series_id}'
            county,pop_id = make_request_with_retries(url)
            if county:
                    counties.append(county)
                    pop_ids.append(pop_id)
                    break
        except Exception as e:  # Catching a general exception for any other errors
            print(f"An error occurred while processing {url}: {e}")
# Print the list to see the result
print(f"Collected {len(counties)} POP Series ID's.")
county_series_ids_df = pd.DataFrame({
    'County': counties,
    'POP ID': pop_ids
})
county_series_ids_df.head(10)

['ANDE', 'ANDR', 'ANGE', 'ARAN', 'ARCH', 'ARMS', 'ATAS', 'AUST', 'BAIL', 'BAND', 'BAST', 'BAYL', 'BEEC', 'BELL', 'BEXA', 'BLAN', 'BORD', 'BOSQ', 'BOWI', 'BRAZ', 'BRAZ', 'BREW', 'BRIS', 'BROO', 'BROW', 'BURL', 'BURN', 'CALD', 'CALH', 'CALL', 'CAME', 'CAMP', 'CARS', 'CASS', 'CAST', 'CHAM', 'CHER', 'CHIL', 'CLAY', 'COCH', 'COKE', 'COLE', 'COLL', 'COLL', 'COLO', 'COMA', 'COMA', 'CONC', 'COOK', 'CORY', 'COTT', 'CRAN', 'CROC', 'CROS', 'CULB', 'DALL', 'DALL', 'DAWS', 'DEAF', 'DELT', 'DENT', 'DEWI', 'DICK', 'DIMM', 'DONL', 'DUVA', 'EAST', 'ECTO', 'EDWA', 'ELLI', 'ELPA', 'ERAT', 'FALL', 'FANN', 'FAYE', 'FISH', 'FLOY', 'FOAR', 'FORT', 'FRAN', 'FREE', 'FRIO', 'GAIN', 'GALV', 'GARZ', 'GILL', 'GLAS', 'GOLI', 'GONZ', 'GRAY', 'GRAY', 'GREG', 'GRIM', 'GUAD', 'HALE', 'HALL', 'HAMI', 'HANS', 'HARD', 'HARD', 'HARR', 'HARR', 'HART', 'HASK', 'HAYS', 'HEMP', 'HEND', 'HIDA', 'HILL', 'HOCK', 'HOOD', 'HOPK', 'HOUS', 'HOWA', 'HUDS', 'HUNT', 'HUTC', 'IRIO', 'JACK', 'JACK', 'JASP', 'JEFF', 'JEFF', 'JIMH', 'JIMW',

' county  Anderson County series id TXANDE1POP'

Request succeeded for https://fred.stlouisfed.org/series/TXANDR3POP


' county  Andrews County series id TXANDR3POP'

Request succeeded for https://fred.stlouisfed.org/series/TXANGE5POP


...

' county  Zapata County series id TXZAPA5POP'

Request succeeded for https://fred.stlouisfed.org/series/TXZAVA7POP


' county  Zavala County series id TXZAVA7POP'

Collected 254 POP Series ID's.


,County,POP ID
0,Anderson County,TXANDE1POP
1,Andrews County,TXANDR3POP
2,Angelina County,TXANGE5POP
3,Aransas County,TXARAN7POP
4,Archer County,TXARCH9POP
5,Armstrong County,TXARMS1POP
6,Atascosa County,TXATAS3POP
7,Austin County,TXAUST5POP
8,Bailey County,TXBAIL7POP
9,Bandera County,TXBAND9POP


In [7]:
# write the county series ids to a file. 

file_path = f"../resources/TX_County_POP_ID.csv"  # Construct file path with .csv extension
county_series_ids_df.to_csv(file_path, index=False)